In [0]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC

In [0]:
## Train SVM on train data and return object of SVM classifier
## kernel parameter = ['linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’]

def Train_SVM(train_data, feature_list=[], SVM_kernel="linear", Regularization=1.0):
  
  train_data.dropna()
  train_data = pd.DataFrame(np.nan_to_num(np.array(train_data)), columns = train_data.columns)
  train_data['Label'] = pd.to_numeric(train_data['Label'], errors='coerce')
  train_data = train_data.dropna(subset=['Label'])
  
  train_features = train_data[feature_list]
  train_labels = train_data['Label']
  train_labels = train_labels.astype('int')
  
  svclassifier = SVC(kernel=SVM_kernel, C=Regularization)
  
  svclassifier.fit(train_features, train_labels)
  
  return svclassifier

In [0]:
## Store trained classifier in a file to reuse in other codes without training again on same data

def Store_Trained_SVM(SVM_obj, Filepath):
  
  with open(Filepath, "wb") as file:
    pickle.dump(SVM_obj, file)

In [0]:
## Load stored trained classifier and returns SVM classifier object

def Load_Trained_SVM(Filepath):
  
  with open(Filepath, "rb") as file:
    SVM_obj = pickle.load(file)

  return SVM_obj

In [0]:
## returns predictions for test data using trained SVM classifer 

def SVM_Predict(svm_obj, test_data, feature_list=[]):
  
  test_features = test_data[feature_list]
  predictions = svm_obj.predict(test_features)
  
  return predictions

In [0]:
def Evaluate_SVM(test_data, SVM_Model_FilePath, feature_list=[]):
  
  test_data.dropna()
  test_data = pd.DataFrame(np.nan_to_num(np.array(test_data)), columns = test_data.columns)
  test_data['Label'] = pd.to_numeric(test_data['Label'], errors='coerce')
  test_data = test_data.dropna(subset=['Label'])
  
  svm_obj = Load_Trained_SVM(SVM_Model_FilePath)
  
  predictions = SVM_Predict(svm_obj, test_data, feature_list)
  
  test_labels = test_data['Label']
  
  test_labels = test_labels.astype('int')
  errors = abs(predictions - test_labels)

  # Calculate mean absolute error (MAE)
  MAE = round(np.mean(errors), 2)
  
  ## Confusion Matrix and Classification Report
  Confusion_Matrix = confusion_matrix(test_labels,predictions)
  Report = classification_report(test_labels,predictions)
  
  
  return MAE, Confusion_Matrix, Report
  

In [0]:
## Execute below code for different embeddings

## Embedded_Data_Train has columns with values for text embedding of post and "Label" column has value 0 or 1
## Test_Embedding_FilePath : Where trained classifier will be saved
## feature_list = colnames which contains values of embeddings (exclude "Label")

Column_List = [ "Caption"]
Vector_Size = 300
Embedding_Cols = [str(i) for i in range(Vector_Size)]
Column_List.extend(Embedding_Cols)
Column_List.append("Label")

Train_Embedding_FilePath = "/content/TrainData_Word2Vec_Embeddings.csv"
Test_Embedding_FilePath = "/content/TestData_Word2Vec_Embeddings.csv"
SVM_Model_FilePath = "/content/SVM_Train_Model.pkl"

train_data = pd.read_csv(Train_Embedding_FilePath, usecols=Column_List)
test_data = pd.read_csv(Test_Embedding_FilePath, usecols=Column_List)

## Training Phase
svm_obj = Train_SVM(train_data, Embedding_Cols)
Store_Trained_SVM(svm_obj, SVM_Model_FilePath)

In [8]:
## Testing Phase
MAE, Confusion_Matrix, Report = Evaluate_SVM(test_data, SVM_Model_FilePath, Embedding_Cols)

print("MEAN ABSOLUTE ERROR: ", MAE)

print("\n")
print("============ CONFUSION MATRIX ===============")
print(Confusion_Matrix)

print("\n")
print("============ CLASSIFICATION REPORT ==============")
print(Report)

MEAN ABSOLUTE ERROR:  0.17


============ CONFUSION MATRIX ===============
[[610 123]
 [127 596]]


============ CLASSIFICATION REPORT ==============
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       733
           1       0.83      0.82      0.83       723

    accuracy                           0.83      1456
   macro avg       0.83      0.83      0.83      1456
weighted avg       0.83      0.83      0.83      1456



In [0]:
## GLove Embeddings

Column_List = [ "Caption_Tokens"]
Vector_Size = 300
Embedding_Cols = [str(i) for i in range(Vector_Size)]
Column_List.extend(Embedding_Cols)
Column_List.append("Label")

Train_Embedding_FilePath = "/content/TrainData_Glove_Embeddings.csv"
Test_Embedding_FilePath = "/content/TestData_Glove_Embeddings.csv"
SVM_Model_FilePath = "/content/SVMG_love_Train_Model.pkl"

train_data = pd.read_csv(Train_Embedding_FilePath, usecols=Column_List)
test_data = pd.read_csv(Test_Embedding_FilePath, usecols=Column_List)

## Training Phase
svm_obj = Train_SVM(train_data, Embedding_Cols)
Store_Trained_SVM(svm_obj, SVM_Model_FilePath)


In [10]:
## Testing Phase
MAE, Confusion_Matrix, Report = Evaluate_SVM(test_data, SVM_Model_FilePath, Embedding_Cols)

print("MEAN ABSOLUTE ERROR: ", MAE)

print("\n")
print("============ CONFUSION MATRIX ===============")
print(Confusion_Matrix)

print("\n")
print("============ CLASSIFICATION REPORT ==============")
print(Report)

MEAN ABSOLUTE ERROR:  0.25


============ CONFUSION MATRIX ===============
[[588 129]
 [235 504]]


============ CLASSIFICATION REPORT ==============
              precision    recall  f1-score   support

           0       0.71      0.82      0.76       717
           1       0.80      0.68      0.73       739

    accuracy                           0.75      1456
   macro avg       0.76      0.75      0.75      1456
weighted avg       0.76      0.75      0.75      1456

